In [3]:
import os, sys, codecs, time, datetime
import fileinput
import os.path
import stanza
from io import StringIO
from subprocess import call


from docopt import docopt
from propsde.applications.viz_tree import DepTreeVisualizer

import propsde.applications.run as run
import sys
if sys.version_info[0] >= 3:
    unicode = str
    
stanza.download('de')
nlp = stanza.Pipeline('de')

2020-05-12 09:56:42 INFO: Downloading default packages for language: de (German)...
2020-05-12 09:56:43 INFO: File exists: /home/chris/stanza_resources/de/default.zip.
2020-05-12 09:56:48 INFO: Finished downloading models and saved to /home/chris/stanza_resources.
2020-05-12 09:56:48 INFO: Loading these models for language: de (German):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| pos       | gsd     |
| lemma     | gsd     |
| depparse  | gsd     |
| ner       | conll03 |

2020-05-12 09:56:48 INFO: Use device: gpu
2020-05-12 09:56:48 INFO: Loading: tokenize
2020-05-12 09:56:51 INFO: Loading: mwt
2020-05-12 09:56:51 INFO: Loading: pos
2020-05-12 09:56:52 INFO: Loading: lemma
2020-05-12 09:56:52 INFO: Loading: depparse
2020-05-12 09:56:53 INFO: Loading: ner
2020-05-12 09:56:53 INFO: Done loading processors!


In [4]:
import pickle

In [5]:
with open('polizeiberichte.pkl', 'rb') as f:
    data = pickle.load(f, encoding="utf8")

Gute Texte: 5
Schwierige Texte: 11

In [173]:
text_example = data[22]["text1"]
print(text_example)

Die Täter versuchten in der Nacht vom 19. zum 20. Juni 2014, die Eingangstür des Bürokomplexes in der Feldtmannstraße gewaltsam zu öffnen. Als dies nicht gelang, hebelten sie schließlich ein Fenster an der Rückseite des Hauses auf und gelangten so in das Objekt. Aus den Büroräumen mehrerer dort ansässiger Firmen entwendeten sie neben Geld auch eine EC-Karte, die bereits am Freitag, 20. Juni 2014, gegen 5.10 Uhr an einem Geldautomaten in der Danziger Straße eingesetzt wurde.


In [174]:
text_example = text_example.replace(". ", ".\n")

In [175]:
print(text_example)

Die Täter versuchten in der Nacht vom 19.
zum 20.
Juni 2014, die Eingangstür des Bürokomplexes in der Feldtmannstraße gewaltsam zu öffnen.
Als dies nicht gelang, hebelten sie schließlich ein Fenster an der Rückseite des Hauses auf und gelangten so in das Objekt.
Aus den Büroräumen mehrerer dort ansässiger Firmen entwendeten sie neben Geld auch eine EC-Karte, die bereits am Freitag, 20.
Juni 2014, gegen 5.10 Uhr an einem Geldautomaten in der Danziger Straße eingesetzt wurde.


In [176]:
file = codecs.open("examples.txt", encoding='utf8')
gs = run.parseSentences(file)

running mate-tools
lemmatizing
parsing + tagging
collapsing
java -jar ext/org.jobimtext.collapsing.jar -i tmp/parsed.conll06 -o tmp -sf -l de -r ext/resources/german_modified.txt -f c -np -nt
UFF


In [177]:
for g,tree in gs: 
    for prop in g.getPropositions('pdf'):
        curProp = '{0}:({1})'.format(prop.pred,
                                      ", ".join([rel + ":" + arg for rel,arg in sorted(prop.args,key=lambda rel:prop.rel_order(rel[0]))]))
        print(curProp)

versuchen:(subj:Die Täter , prep_in:der Nacht vom 19 )
Juni 2014:(dobj:die Eingangstür des Bürokomplexes in der Feldtmannstraße gewaltsam zu öffnen )
gelangt:(subj:sie , prep_in:das Objekt , mod:so )
aufhebeln:(subj:sie , dobj:ein Fenster an der Rückseite des Hauses , mod:Als dies nicht gelang , mod:schließlich )
und:(conj_und:Als dies nicht gelang , hebelten sie schließlich ein Fenster an der Rückseite des Hauses auf , conj_und:sie gelangten so in das Objekt )
entwenden:(subj:sie , dobj:auch eine EC-Karte , die bereits an Freitag , 20 , prep_aus:den Büroräumen mehrerer dort ansässiger Firmen , prep_neben:Geld )
20:(subj:auch eine EC-Karte , prep_an:Freitag , mod:bereits )
einsetzen:(obj:Juni 2014 , prep_gegen:5.10 Uhr , prep_an:einem Geldautomaten in der Danziger Straße )


In [178]:
prop = g.getPropositions('pdf')[0]
print(prop.args)
print(prop.pred)

[['obj', 'Juni 2014 '], ['prep_gegen', '5.10 Uhr '], ['prep_an', 'einem Geldautomaten in der Danziger Straße ']]
einsetzen


In [179]:
test = gs[0][0]
print(test)

Die Täter versuchten in der Nacht vom 19 .
519	2,Täter	NN	0	0	subj,520
520	3,versuchten	VVFIN	1	1	
522	6,Nacht	NN	0	0	prep_in,520
602	7,vom;8,19		0	0	mod,522



In [180]:
from itertools import compress

def _map_article_of_subj_to_pronoun(arg):
    personal_pronouns = {"der": "er", "das": "es", "die": "sie", "dem": "er", "den": "sie"}
    
    if "subj" in arg:
        words = arg[1].lower().split()
        pronoun_place = list(word in personal_pronouns for word in words)
        pronoun = list(compress(words, pronoun_place))
    
        if len(pronoun) == 1:
            current_subj = (personal_pronouns[pronoun[0]], arg[1])
            return current_subj
    return None

def _get_core_from_noun_phrase(noun_phrase):
    pos_tag = nlp(noun_phrase)
    for sent in pos_tag.sentences:
        for word in sent.words:
            if word.head == 0:
                return {"np": noun_phrase, "core:":word.lemma}



In [192]:
subjects = []
nouns = []
prop_cache = []
index = 0

for i, (g, tree) in enumerate(gs):
    for prop in g.getPropositions('pdf'):
        subjects.append([])
        prop_cache.append({"args": prop.args, "pred": prop.pred})
        
while index < len(prop_cache):
    for i, arg in enumerate(prop_cache[index]["args"]):
        words = arg[1].lower().split()

        _article_pronoun = _map_article_of_subj_to_pronoun(arg)
        _noun_phrase_core = _get_core_from_noun_phrase(arg[1])
        if _article_pronoun:
            subjects[i].append(_article_pronoun)
        if _noun_phrase_core:
            nouns.append(_noun_phrase_core)
        if _noun_phrase_core in nouns:
            #prop_cache[index]["args"][i][1] = {"core": _noun_phrase_core, "np": arg[1]}
            prop_cache[index]["args"][i][1] = _noun_phrase_core
        for j in range(1, len(subjects)):
            try:
                if index > 0 and len(subjects[index-j]) > 0 and subjects[index-j][0][0] in words:
                    prop_cache[index]["args"][i][1] = _get_core_from_noun_phrase(subjects[index-j][0][1])
                    pass
            except:
                print("Error")
    index += 1
    

8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8


Der regierende Bürgermeister fährt zum Rathaus.
Im roten Rathaus geht er in den Sitzungssaal.
Der alte Sitzungssaal liegt neben dem Büro von Michael Müller.

In [191]:
for prop in prop_cache:
    print(prop)
    

{'args': [['subj', {'np': 'Die Täter ', 'core:': 'Täter'}], ['prep_in', {'np': 'der Nacht vom 19 ', 'core:': 'Nacht'}]], 'pred': 'versuchen'}
{'args': [['dobj', {'np': 'die Eingangstür des Bürokomplexes in der Feldtmannstraße gewaltsam zu öffnen ', 'core:': 'öffnen'}]], 'pred': 'Juni 2014'}
{'args': [['subj', {'np': 'Die Täter ', 'core:': 'Täter'}], ['mod', {'np': 'so ', 'core:': 'so'}], ['prep_in', {'np': 'das Objekt ', 'core:': 'Objekt'}]], 'pred': 'gelangt'}
{'args': [['mod', {'np': 'Als dies nicht gelang ', 'core:': 'gelingen'}], ['subj', {'np': 'Die Täter ', 'core:': 'Täter'}], ['mod', {'np': 'schließlich ', 'core:': 'schließlich'}], ['dobj', {'np': 'ein Fenster an der Rückseite des Hauses ', 'core:': 'Fenster'}]], 'pred': 'aufhebeln'}
{'args': [['conj_und', {'np': 'Die Täter ', 'core:': 'Täter'}], ['conj_und', {'np': 'Die Täter ', 'core:': 'Täter'}]], 'pred': 'und'}
{'args': [['prep_aus', {'np': 'den Büroräumen mehrerer dort ansässiger Firmen ', 'core:': 'Büroraum'}], ['subj', {'

In [172]:
nlp("mehrere")

[
  [
    {
      "id": "1",
      "text": "mehrere",
      "lemma": "mehrere",
      "upos": "PRON",
      "xpos": "PIS",
      "feats": "Case=Nom|Definite=Ind|Number=Plur|PronType=Ind",
      "head": 0,
      "deprel": "root",
      "misc": "start_char=0|end_char=7"
    }
  ]
]

In [93]:
for j in range(5, 0, -1):
    print(j)

5
4
3
2
1
